In [1]:
import pandas as pd  # Fixed: was "import pd"
import json
import glob
import os
import numpy as np
from datetime import datetime
import sys

# --- Setup Paths ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

import pandas as pd
import json
import glob
import numpy as np
from datetime import datetime

# פונקציית עזר לשליפה בטוחה של נתונים מקוננים
def safe_get(d, *keys, default='N/A'):
    for key in keys:
        if isinstance(d, dict):
            d = d.get(key, {})
        else:
            return default
    return d if (d != {} and d is not None) else default

def safe_float(value, default=0.0):
    if value is None or value == 'N/A': return default
    try:
        if isinstance(value, str):
            value = value.replace('%', '').strip()
        return float(value)
    except: return default

# ============================================================
# 1. טעינת קבצים וקיבוץ לפי Run_ID
# ============================================================
DERIV_ROOT = os.path.join(PROJECT_ROOT, "derivatives")
json_files = glob.glob(os.path.join(DERIV_ROOT, "*", "*.json"))

from collections import defaultdict
runs = defaultdict(dict)

for json_path in json_files:
    filename = os.path.basename(json_path)
    run_id = filename.split('__')[0]
    
    with open(json_path, 'r', encoding='utf-8') as f:  # Added encoding
        data = json.load(f)
    
    # מיון לקטגוריות לפי תוכן הקובץ
    if 'loader_report' in filename or 'step01' in filename:
        runs[run_id]['step_01'] = data
    elif 'preprocess_summary' in filename:
        runs[run_id]['step_02'] = data
    elif 'filtering_summary' in filename:
        runs[run_id]['step_04'] = data
    elif 'reference' in filename and 'summary' in filename:
        runs[run_id]['step_05'] = data
    elif 'kinematics_summary' in filename:
        runs[run_id]['step_06'] = data

print(f"📊 Found data for {len(runs)} run(s)")
print(f"📁 Steps loaded per run:")
for rid, steps in runs.items():
    print(f"  {rid}: {list(steps.keys())}")
print()

# ============================================================
# 2. בניית טבלת המאסטר (Master Audit)
# ============================================================
all_summaries = []
for run_id, steps in runs.items():
    s01 = steps.get('step_01', {})
    s02 = steps.get('step_02', {})
    s04 = steps.get('step_04', {})
    s05 = steps.get('step_05', {})
    s06 = steps.get('step_06', {})
    
    if not s01 or not s06:
        print(f"⚠️  Skipping {run_id}: Missing critical step data")
        continue
    
    fps = safe_float(safe_get(s01, 'raw_data_quality', 'sampling_rate_actual'), default=120.0)
    
    row = {
        # --- Identity ---
        "Run_ID": run_id,
        "Processing_Date": safe_get(s01, 'identity', 'processing_timestamp'),
        "OptiTrack_Error_mm": safe_float(safe_get(s01, 'raw_data_quality', 'optitrack_mean_error_mm')),
        
        # --- Raw Quality & Preprocess (Step 01 & 02) ---
        "Total_Frames": safe_get(s01, 'raw_data_quality', 'total_frames', default=0),
        "Missing_Raw_%": safe_float(safe_get(s02, 'raw_missing_percent')),
        "Max_Gap_Frames": safe_get(s02, 'max_interpolation_gap', default=0),
        "Max_Gap_MS": round((safe_float(safe_get(s02, 'max_interpolation_gap')) / fps) * 1000, 2),
        
        # --- Bone QC (Step 02) ---
        "Bone_Stability_CV": safe_float(safe_get(s02, 'bone_qc_mean_cv')),
        "Skeletal_Alerts": safe_get(s02, 'bone_qc_alerts', default=0),
        "Worst_Bone": safe_get(s02, 'worst_bone'),
        
        # --- Reference Stability (Step 05) ---
        "Ref_Stability_mm": safe_float(safe_get(s05, 'reference_metrics', 'ref_stability_mm')),
        "Ref_Status": safe_get(s05, 'reference_metrics', 'ref_quality_status'),
        
        # --- Signal Quality (Step 06) ---
        "Signal_Noise_RMS": safe_float(safe_get(s06, 'signal_quality', 'avg_vel_residual_rms')),
        "Dom_Freq_Hz": safe_float(safe_get(s06, 'signal_quality', 'avg_dominant_freq_hz')),
        "Quat_Norm_Error": safe_float(safe_get(s06, 'signal_quality', 'max_quat_norm_error')),
        
        # --- Kinematics (Step 06) ---
        "Max_Ang_Vel": safe_float(safe_get(s06, 'metrics', 'angular_velocity', 'max')),
        "Mean_Ang_Vel": safe_float(safe_get(s06, 'metrics', 'angular_velocity', 'mean')),
        "Max_Lin_Acc": safe_float(safe_get(s06, 'metrics', 'linear_accel', 'max')),
        "Outlier_Frames": safe_get(s06, 'effort_metrics', 'outlier_frame_count', default=0),
        
        # --- Effort Metrics (Step 06) ---
        "Path_Length_M": round(safe_float(safe_get(s06, 'effort_metrics', 'total_path_length_mm')) / 1000, 2),
        "Intensity_Index": safe_float(safe_get(s06, 'effort_metrics', 'intensity_index')),
        
        # --- Overall Status ---
        "Pipeline_Status": safe_get(s06, 'overall_status'),
    }
    
    # === לוגיקת ציון איכות משודרגת ===
    score = 100.0
    # קנסות על איכות נתונים
    score -= safe_float(row["Missing_Raw_%"]) * 5
    score -= (safe_float(row["Max_Gap_MS"]) / 10) # קנס על חורים גדולים
    
    # קנסות על יציבות שלד (Bone QC)
    score -= safe_float(row["Bone_Stability_CV"]) * 10 
    score -= safe_float(row["Skeletal_Alerts"]) * 5
    
    # קנס על יציבות רפרנס
    ref_stab = safe_float(row["Ref_Stability_mm"])
    if ref_stab > 4.0: score -= 15
    
    row["Quality_Score"] = round(max(0, min(100, score)), 2)
    
    # === החלטה מחקרית ===
    # תנאי סף מחמירים ל-ACCEPT
    if (row["Pipeline_Status"] == "PASS" and 
        row["Quality_Score"] >= 75 and 
        row["Ref_Status"] == "PASS" and 
        safe_float(row["Bone_Stability_CV"]) < 1.5):
        row["Research_Decision"] = "ACCEPT"
    elif row["Pipeline_Status"] == "PASS" and row["Quality_Score"] >= 50:
        row["Research_Decision"] = "REVIEW"
    else:
        row["Research_Decision"] = "REJECT"
    
    all_summaries.append(row)
# ============================================================
# 3. יצירת DataFrame וייצוא לאקסל
# ============================================================
if not all_summaries:
    print("❌ No complete runs found to aggregate!")
else:
    df_master = pd.DataFrame(all_summaries)
    df_master = df_master.sort_values('Quality_Score', ascending=False).reset_index(drop=True)
    
    REPORTS_DIR = os.path.join(PROJECT_ROOT, "reports")
    os.makedirs(REPORTS_DIR, exist_ok=True)
    excel_path = os.path.join(REPORTS_DIR, f"Master_Audit_Log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx")
    
    with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
        df_master.to_excel(writer, index=False, sheet_name='Audit_Log')
        workbook = writer.book
        worksheet = writer.sheets['Audit_Log']
        
        # Header format
        header_fmt = workbook.add_format({'bold': True, 'bg_color': '#4472C4', 'font_color': 'white'})
        for col_num, value in enumerate(df_master.columns.values):
            worksheet.write(0, col_num, value, header_fmt)
        
        # Conditional formatting for Research_Decision
        red_fmt = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
        yellow_fmt = workbook.add_format({'bg_color': '#FFEB9C', 'font_color': '#9C6500'})
        green_fmt = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
        
        col_idx = df_master.columns.get_loc("Research_Decision")
        for row_num in range(1, len(df_master) + 1):
            decision = df_master.iloc[row_num-1]['Research_Decision']
            if decision == 'ACCEPT':
                worksheet.write(row_num, col_idx, decision, green_fmt)
            elif decision == 'REVIEW':
                worksheet.write(row_num, col_idx, decision, yellow_fmt)
            else:
                worksheet.write(row_num, col_idx, decision, red_fmt)
        
        # Auto-fit columns
        for i, col in enumerate(df_master.columns):
            max_len = max(df_master[col].astype(str).str.len().max(), len(col))
            worksheet.set_column(i, i, min(max_len + 2, 40))
    
    print(f"\n{'='*70}")
    print(f"🎉 Master Audit Log Created")
    print(f"{'='*70}")
    print(f"📊 Total Runs: {len(all_summaries)}")
    print(f"💾 File: {excel_path}")
    print(f"{'='*70}\n")
    
    print("Decision Summary:")
    print(df_master['Research_Decision'].value_counts())
    
    print("\nQuality Score Stats:")
    print(f"  Mean: {df_master['Quality_Score'].mean():.2f}")
    print(f"  Min:  {df_master['Quality_Score'].min():.2f}")
    print(f"  Max:  {df_master['Quality_Score'].max():.2f}")
    
    print("\nPreview:")
    display(df_master.head())

📊 Found data for 2 run(s)
📁 Steps loaded per run:
  734_T1_P2_R1_Take 2025-12-01 02.28.24 PM: ['step_01', 'step_02', 'step_04', 'step_05', 'step_06']
  763_T2_P2_R2_Take_2025-12-25 10.51.23 AM_005: ['step_01', 'step_02', 'step_04', 'step_05', 'step_06']


🎉 Master Audit Log Created
📊 Total Runs: 2
💾 File: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\reports\Master_Audit_Log_20260113_211719.xlsx

Decision Summary:
Research_Decision
ACCEPT    2
Name: count, dtype: int64

Quality Score Stats:
  Mean: 87.89
  Min:  87.88
  Max:  87.91

Preview:


,Run_ID,Processing_Date,OptiTrack_Error_mm,Total_Frames,Missing_Raw_%,Max_Gap_Frames,Max_Gap_MS,Bone_Stability_CV,Skeletal_Alerts,Worst_Bone,...,Quat_Norm_Error,Max_Ang_Vel,Mean_Ang_Vel,Max_Lin_Acc,Outlier_Frames,Path_Length_M,Intensity_Index,Pipeline_Status,Quality_Score,Research_Decision
0,763_T2_P2_R2_Take_2025-12-25 10.51.23 AM_005,2026-01-12 15:09,0.0,17263,0.0,10,83.33,0.376,0,Hips->Spine,...,0.0,1359.12,113.83,44376.02,40,61.28,0.291,PASS,87.91,ACCEPT
1,734_T1_P2_R1_Take 2025-12-01 02.28.24 PM,2026-01-13 20:35,0.0,19617,0.0,10,83.33,0.379,0,Hips->Spine,...,0.0,900.17,44.92,19427.84,0,35.93,0.386,PASS,87.88,ACCEPT
